In [2]:
import pandas as pd
import numpy as np

In [3]:
import xgboost as xgb

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv("/content/drive/My Drive/data/heart_attack_prediction_dataset.csv")
df.head()

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


In [7]:
df = df.drop(columns = ["Patient ID", "Continent", "Hemisphere"])
df[['systolic', 'diastolic']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
df = df.drop(columns = ["Blood Pressure"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              8763 non-null   int64  
 1   Sex                              8763 non-null   object 
 2   Cholesterol                      8763 non-null   int64  
 3   Heart Rate                       8763 non-null   int64  
 4   Diabetes                         8763 non-null   int64  
 5   Family History                   8763 non-null   int64  
 6   Smoking                          8763 non-null   int64  
 7   Obesity                          8763 non-null   int64  
 8   Alcohol Consumption              8763 non-null   int64  
 9   Exercise Hours Per Week          8763 non-null   float64
 10  Diet                             8763 non-null   object 
 11  Previous Heart Problems          8763 non-null   int64  
 12  Medication Use      

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
columns_to_encode = ["Sex", "Diet", "Country"]
for col in columns_to_encode:
  df[col] = encoder.fit_transform(df[col])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              8763 non-null   int64  
 1   Sex                              8763 non-null   int64  
 2   Cholesterol                      8763 non-null   int64  
 3   Heart Rate                       8763 non-null   int64  
 4   Diabetes                         8763 non-null   int64  
 5   Family History                   8763 non-null   int64  
 6   Smoking                          8763 non-null   int64  
 7   Obesity                          8763 non-null   int64  
 8   Alcohol Consumption              8763 non-null   int64  
 9   Exercise Hours Per Week          8763 non-null   float64
 10  Diet                             8763 non-null   int64  
 11  Previous Heart Problems          8763 non-null   int64  
 12  Medication Use      

In [9]:
X = df.drop(columns = ["Heart Attack Risk"])
y = df["Heart Attack Risk"]

**XGBOOST**

In [131]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43, stratify = y)

In [132]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [133]:
from sklearn.metrics import accuracy_score

In [134]:
parameters_grid = {
    'max_depth': [2, 4, 6, 8, 10],
    'learning_rate': [0.01, 0.02, 0.04, 0.06, 0.08, 1.0],
    'n_estimators': [10, 20, 30, 40, 50, 60, 70 , 80 , 90],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
}


**USING RANDOMIZEDSEARCHCV**

In [135]:
xgb = xgb.XGBClassifier()
r_xgb = RandomizedSearchCV(xgb, parameters_grid, cv=5, scoring='accuracy', n_iter=10, random_state = 42, verbose = 3)

In [150]:
r_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.636 total time=  12.1s
[CV 2/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.635 total time=   2.7s
[CV 3/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.637 total time=   0.6s
[CV 4/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.629 total time=   0.6s
[CV 5/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.638 total time=   0.6s
[CV 1/5] END colsample_bytree=0.5, learning_rate=0.06, max_depth=2, n_estimators=90, subsample=0.7;, score=0.642 total time=   0.1s
[CV 2/5] END colsample_bytree=0.5, learning_rate=0.06, max_depth=2, n_estimators=90, subsample=0.7;, score=0.642 total time=   0.1s
[CV 3/5] E

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   param_distributions={'colsample_bytree': [0.5, 0.6, 0.7, 0.8,
                                                             0.9, 1.0],
                                        'learning_rate': [0.01, 0.02, 0.04,
                                                          0.06, 0.08, 1.0],
                                        'max_depth': [2, 4, 6, 8, 10],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9,
                                                      1.0]},
                   random_state=42, scoring='accuracy', verbose=3)

In [153]:
predictions = r_xgb.predict(X_test)

In [154]:
print(f"Accuracy: {accuracy_score(y_test, predictions)}")

Accuracy: 0.6417569880205363


In [155]:
print("Best parameters found: ", r_xgb.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(r_xgb.best_score_)))

Best parameters found:  {'subsample': 0.9, 'n_estimators': 20, 'max_depth': 4, 'learning_rate': 0.06, 'colsample_bytree': 0.8}
Lowest RMSE found:  0.8012116359564503


In [156]:
from sklearn.metrics import confusion_matrix, classification_report
print(f"Confusion Matrix \n {confusion_matrix(y_test, predictions)}")
print(f"Classification Report \n {classification_report(y_test, predictions)}")

Confusion Matrix 
 [[1125    0]
 [ 628    0]]
Classification Report 
               precision    recall  f1-score   support

           0       0.64      1.00      0.78      1125
           1       0.00      0.00      0.00       628

    accuracy                           0.64      1753
   macro avg       0.32      0.50      0.39      1753
weighted avg       0.41      0.64      0.50      1753



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [157]:
df["Heart Attack Risk"].value_counts()

Heart Attack Risk
0    5624
1    3139
Name: count, dtype: int64

The model is performing poorly on the 1 class, due to class imbalace. Using accuracy as a scoring metric can be  misleading as an high accuracy does not signify the performance of the model on both classes as seen above.
Scoring will be set to f1 to select the best model



**USING F1 AS THE SCORING METRIC IN THE RANDOMIZEDSEARCHCV**

In [151]:
xgb_f1 = RandomizedSearchCV(xgb, parameters_grid, cv=5, scoring='f1', n_iter=10, random_state = 42, verbose = 3)

In [152]:
xgb_f1.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.069 total time=  14.0s
[CV 2/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.048 total time=   1.4s
[CV 3/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.045 total time=   0.5s
[CV 4/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.037 total time=   0.6s
[CV 5/5] END colsample_bytree=0.9, learning_rate=0.04, max_depth=10, n_estimators=60, subsample=0.9;, score=0.052 total time=   0.6s
[CV 1/5] END colsample_bytree=0.5, learning_rate=0.06, max_depth=2, n_estimators=90, subsample=0.7;, score=0.000 total time=   0.1s
[CV 2/5] END colsample_bytree=0.5, learning_rate=0.06, max_depth=2, n_estimators=90, subsample=0.7;, score=0.000 total time=   0.1s
[CV 3/5] E

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   param_distributions={'colsample_bytree': [0.5, 0.6, 0.7, 0.8,
                                                             0.9, 1.0],
                                        'learning_rate': [0.01, 0.02, 0.04,
                                                          0.06, 0.08, 1.0],
                                        'max_depth': [2, 4, 6, 8, 10],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9,
                                                      1.0]},
                   random_state=42, scoring='f1', verbose=3)

In [158]:
xgb_f1_predictions = xgb_f1.predict(X_test)

In [159]:
print(f"Accuracy: {accuracy_score(y_test, xgb_f1_predictions)}")

Accuracy: 0.5658870507701084


In [160]:
print("Best parameters found: ", xgb_f1.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(xgb_f1.best_score_)))

Best parameters found:  {'subsample': 0.6, 'n_estimators': 10, 'max_depth': 4, 'learning_rate': 1.0, 'colsample_bytree': 0.8}
Lowest RMSE found:  0.5225968446852519


In [161]:
from sklearn.metrics import confusion_matrix, classification_report
print(f"Confusion Matrix \n {confusion_matrix(y_test, xgb_f1_predictions)}")
print(f"Classification Report \n {classification_report(y_test, xgb_f1_predictions)}")

Confusion Matrix 
 [[857 268]
 [493 135]]
Classification Report 
               precision    recall  f1-score   support

           0       0.63      0.76      0.69      1125
           1       0.33      0.21      0.26       628

    accuracy                           0.57      1753
   macro avg       0.48      0.49      0.48      1753
weighted avg       0.53      0.57      0.54      1753

